<p align="center"><h1 align="center">Flower Image Classification with Keras</h1> <h3 align="center">(Prepare to deploy model and preprocessor to REST API/Web Dashboard in four easy steps...)</h3></p>
<p align="center"><img width="80%" src='https://drive.google.com/thumbnail?id=1ea5R66cqAXs3g4oIeEiUUMhictrL2IBg&sz=w1200-h1200' /></p>


---



## **(1) Train Your Model**

In [1]:
# Load a pretrained Keras model.
!wget https://s3.amazonaws.com/aimodelshare.models/keras_flowermodel.h5

import tensorflow as tf

model = tf.keras.models.load_model('keras_flowermodel.h5')

--2020-10-27 02:00:11--  https://s3.amazonaws.com/aimodelshare.models/keras_flowermodel.h5
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.217.11.22
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.217.11.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 60081120 (57M) [application/x-www-form-urlencoded]
Saving to: ‘keras_flowermodel.h5’

keras_flowermodel.h 100%[===================>]  57.30M  15.8MB/s    in 4.5s    

2020-10-27 02:00:17 (12.8 MB/s) - ‘keras_flowermodel.h5’ saved [60081120/60081120]



In [2]:
# Keras model expects shape with samples, height, width, and channels last.
model.input_shape

(None, 192, 192, 3)

## **(2) Save Model to ONNX**

In [ ]:
# Load libraries for ONNX model conversion (Keras to ONNX).
! pip3 install keras2onnx
! pip3 install onnxruntime

In [8]:
# Save model to onnx file.

import os
os.environ['TF_KERAS'] = '1' # Add this environmental variable whenever you use TensorFlow's `tf.keras` to build your Keras model.

import onnx
import keras2onnx

# Convert model to onnx object.
import onnx
from keras2onnx import convert_keras # Specific to your architecture.
onnx_model = convert_keras(model, 'my_model.onnx')

# Save model to local .onnx file.
with open("my_model.onnx", "wb") as f:
    f.write(onnx_model.SerializeToString())

tf executing eager_mode: True
tf.keras model eager_mode: False
The ONNX operator number change on the optimization: 49 -> 24


## **(3) Write Preprocessor Function**

> ### Preprocessor functions for image prediction models can use ***`cv2`*** and ***`numpy`*** to read in and preprocess images. 

In [3]:
# Write preprocessor that will match up with model's expected input shape.
def preprocessor(data, shape=(192, 192)):
        """
        This function reads in images, resizes them to a fixed shape, and
        min/max transforms them, before converting feature values to float32
        for ONNX.
        
        params:
            data
                list of unprocessed images
                      
        returns:
            X
                numpy array of preprocessed image data
                  
        """
           
        import cv2
        import numpy as np

        "Resize a color image and min/max transform the image"
        img = cv2.imread(data) # Read in image from filepath.
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) # cv2 reads in images in order of blue green and red, we reverse the order for ML.
        img = cv2.resize(img, shape) # Change height and width of image.
        img = img / 255.0 # Min-max transform.


        # Resize the images.
        X = np.array(img)
        X = np.expand_dims(X, axis=0) # Expand dims to add "1" to object shape [1, h, w, channels].
        X = np.array(X, dtype=np.float32) # Final shape for onnx runtime.
        return X

You may want to verify that your `preprocessor` is working as intended.

In [4]:
# Load in flower_photos.
import pathlib
dataset_url = 'http://download.tensorflow.org/example_images/flower_photos.tgz'
data_dir = tf.keras.utils.get_file(origin=dataset_url, 
                                   fname='flower_photos', 
                                   untar=True)
data_dir = pathlib.Path(data_dir)

228818944/228813984 [==============================] - 1s 0us/step


In [5]:
# Test the first roses image.
roses = list(data_dir.glob('roses/*'))

import os
preprocessor(os.fspath(roses[0])).shape # os.fspath converts PosixPath to simple str.

(1, 192, 192, 3)

## **(4) Save Preprocessor Function**

In [ ]:
# ! pip3 install aimodelshare

In [6]:
def export_preprocessor(preprocessor_function, filepath):
    import dill
    with open(filepath, 'wb') as f:
        dill.dump(preprocessor_function, f)

# import aimodelshare as ai # Once we can deploy this, we use it in lieu of the below.
# ai.export_preprocessor(preprocessor, "preprocessor.pkl")

export_preprocessor(preprocessor, 'preprocessor.pkl')